In [1]:
import os 
import json
import pandas as pd
import traceback

In [3]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

c:\Hardik\Ai_tools\mcq_with_python3.10\MCQ_Generator\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
key = os.getenv("GOOGLE_API_KEY")

In [9]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=key)

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [69]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone,
Make sure the question are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs 
dont put any thing before the resonse only answer in curly bracket and all the response should be in single quote
{response_json} 

"""

In [70]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [71]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [72]:
TEMPLATE2 = """
You are an expert english grammarian and writer . Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz.Only use at max 50 words for complexity
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities 
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""
    

In [73]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [74]:
review_chain = LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [75]:
from langchain.chains import SequentialChain

In [76]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                                        output_variables=["quiz","review"],verbose=True)

In [77]:
file="C:\Hardik\Ai_tools\mcq_with_python3.10\MCQ_Generator\data.txt"

In [78]:
with open(file,'r') as file:
    TEXT = file.read()

In [79]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [80]:
from langchain.callbacks import get_openai_callback

In [81]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
        "text": TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [82]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [83]:
quiz=response.get("quiz")

In [84]:
print(quiz)

{"1": {"mcq": "Who coined the term 'machine learning'?", "options": {"a": "Donald Hebb", "b": "Arthur Samuel", "c": "Walter Pitts", "d": "Warren McCulloch"}, "correct": "b"}, "2": {"mcq": "Which of the following is NOT a synonym for machine learning used in the 1950s?", "options": {"a": "Self-teaching computers", "b": "Cognitive computing", "c": "Artificial intelligence", "d": "Machine cognition"}, "correct": "b"}, "3": {"mcq": "What was the purpose of the 'goof' button in Raytheon Company's 'learning machine'?", "options": {"a": "To reset the machine's memory", "b": "To allow the human operator to correct the machine's errors", "c": "To switch the machine to a different learning mode", "d": "To prevent the machine from overfitting"}, "correct": "b"}, "4": {"mcq": "Who provided a widely quoted definition of machine learning algorithms?", "options": {"a": "Arthur Samuel", "b": "Tom M. Mitchell", "c": "Donald Hebb", "d": "Walter Pitts"}, "correct": "b"}, "5": {"mcq": "Modern-day machine 

In [86]:
quiz = json.loads(quiz)

In [93]:
quiz_table_data = []

for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]  # Corrected indentation
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

# Print the quiz_table_data to verify the result
for data in quiz_table_data:
    print(data)


{'MCQ': "Who coined the term 'machine learning'?", 'Choices': 'a: Donald Hebb | b: Arthur Samuel | c: Walter Pitts | d: Warren McCulloch', 'Correct': 'b'}
{'MCQ': 'Which of the following is NOT a synonym for machine learning used in the 1950s?', 'Choices': 'a: Self-teaching computers | b: Cognitive computing | c: Artificial intelligence | d: Machine cognition', 'Correct': 'b'}
{'MCQ': "What was the purpose of the 'goof' button in Raytheon Company's 'learning machine'?", 'Choices': "a: To reset the machine's memory | b: To allow the human operator to correct the machine's errors | c: To switch the machine to a different learning mode | d: To prevent the machine from overfitting", 'Correct': 'b'}
{'MCQ': 'Who provided a widely quoted definition of machine learning algorithms?', 'Choices': 'a: Arthur Samuel | b: Tom M. Mitchell | c: Donald Hebb | d: Walter Pitts', 'Correct': 'b'}
{'MCQ': 'Modern-day machine learning has which two main objectives?', 'Choices': 'a: Data classification and p

In [94]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Donald Hebb | b: Arthur Samuel | c: Walter Pitts | d: Warren McCulloch',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT a synonym for machine learning used in the 1950s?',
  'Choices': 'a: Self-teaching computers | b: Cognitive computing | c: Artificial intelligence | d: Machine cognition',
  'Correct': 'b'},
 {'MCQ': "What was the purpose of the 'goof' button in Raytheon Company's 'learning machine'?",
  'Choices': "a: To reset the machine's memory | b: To allow the human operator to correct the machine's errors | c: To switch the machine to a different learning mode | d: To prevent the machine from overfitting",
  'Correct': 'b'},
 {'MCQ': 'Who provided a widely quoted definition of machine learning algorithms?',
  'Choices': 'a: Arthur Samuel | b: Tom M. Mitchell | c: Donald Hebb | d: Walter Pitts',
  'Correct': 'b'},
 {'MCQ': 'Modern-day machine learning has which two main objectives?',
  'Choices': 'a

In [97]:
main = pd.DataFrame(quiz_table_data)

In [98]:
main.to_csv("machinelearning.csv",index=False)